In [247]:
%pip install scikit-fuzzy matplotlib scipy networkx

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [248]:
import pandas as pd
import numpy as np
import skfuzzy as fuzz
from skfuzzy import control as ctrl
import matplotlib.pyplot as plt

In [249]:
def cria_df_programa(column_perc):
# Assumindo que o arquivo CSV está na mesma pasta
    df = pd.read_csv("base/MunicipioBrasil_20230102.csv")
    df_programa = df[["cod_mun", "nom_mun", column_perc]].copy()
    df_programa[column_perc] = df_programa[column_perc].str.replace(',', '.')
    df_programa[column_perc] = df_programa[column_perc].astype(float)
    df_programa = df_programa.dropna()
    return df_programa

# Aplicando Fuzzy em "perc_dom_pobres_var1"

In [250]:
def aplica_fuzzy_perc_dom_pobres_var1(df):
    # Variável de entrada
    perc_dom_pobres = ctrl.Antecedent(np.arange(0, 101, 1), 'perc_dom_pobres')
    # Variável de saída
    eligibilidade = ctrl.Consequent(np.arange(0, 101, 1), 'eligibilidade')

    # Definindo os conjuntos fuzzy para a variável de entrada
    perc_dom_pobres['baixissima'] = fuzz.trimf(perc_dom_pobres.universe, [0, 0, 20])
    perc_dom_pobres['baixa'] = fuzz.trimf(perc_dom_pobres.universe, [0, 20, 40])
    perc_dom_pobres['Mediana'] = fuzz.trimf(perc_dom_pobres.universe, [0, 40, 60])
    perc_dom_pobres['Alta'] = fuzz.trimf(perc_dom_pobres.universe, [0, 60, 70])
    perc_dom_pobres['Altíssima'] = fuzz.trimf(perc_dom_pobres.universe, [0, 70, 100])

    # Definindo os conjuntos fuzzy para a variável de saída
    eligibilidade['baixissima'] = fuzz.trimf(eligibilidade.universe, [0, 0, 20])
    eligibilidade['baixa'] = fuzz.trimf(eligibilidade.universe, [0, 20, 40])
    eligibilidade['Mediana'] = fuzz.trimf(eligibilidade.universe, [0, 40, 60])
    eligibilidade['Alta'] = fuzz.trimf(eligibilidade.universe, [0, 60, 70])
    eligibilidade['Altíssima'] = fuzz.trimf(eligibilidade.universe, [0, 70, 100])

    rule1 = ctrl.Rule(perc_dom_pobres['baixissima'], eligibilidade['baixissima'])
    rule2 = ctrl.Rule(perc_dom_pobres['baixa'], eligibilidade['baixa'])
    rule3 = ctrl.Rule(perc_dom_pobres['Mediana'], eligibilidade['Mediana'])
    rule4 = ctrl.Rule(perc_dom_pobres['Alta'], eligibilidade['Alta'])
    rule5 = ctrl.Rule(perc_dom_pobres['Altíssima'], eligibilidade['Altíssima'])

    ctrl_sys = ctrl.ControlSystem([rule1, rule2, rule3, rule4, rule5])
    sim = ctrl.ControlSystemSimulation(ctrl_sys)

    list_ = []
    for index, row in df.iterrows():
        sim.input['perc_dom_pobres'] = row['perc_dom_pobres_var1']
        sim.compute()
        list_.append({'codigo_municipio': f"{row['cod_mun']:.0f}", 'nome_municipio': f"{row['nom_mun']}", 'Percentual_elegibilidade': f"{sim.output['eligibilidade']:.2f}"})
    return list_

In [251]:
perc_dom_pobres = cria_df_programa('perc_dom_pobres_var1')

In [252]:
list_mun_elegibilidade = aplica_fuzzy_perc_dom_pobres_var1(perc_dom_pobres)
print(list_mun_elegibilidade)

[{'codigo_municipio': '1100015', 'nome_municipio': "Alta Floresta D'Oeste", 'Percentual_elegibilidade': '55.67'}, {'codigo_municipio': '1100023', 'nome_municipio': 'Ariquemes', 'Percentual_elegibilidade': '50.29'}, {'codigo_municipio': '1100031', 'nome_municipio': 'Cabixi', 'Percentual_elegibilidade': '55.68'}, {'codigo_municipio': '1100049', 'nome_municipio': 'Cacoal', 'Percentual_elegibilidade': '45.78'}, {'codigo_municipio': '1100056', 'nome_municipio': 'Cerejeiras', 'Percentual_elegibilidade': '54.36'}, {'codigo_municipio': '1100064', 'nome_municipio': 'Colorado do Oeste', 'Percentual_elegibilidade': '50.88'}, {'codigo_municipio': '1100072', 'nome_municipio': 'Corumbiara', 'Percentual_elegibilidade': '56.66'}, {'codigo_municipio': '1100080', 'nome_municipio': 'Costa Marques', 'Percentual_elegibilidade': '56.49'}, {'codigo_municipio': '1100098', 'nome_municipio': "Espig�o D'Oeste", 'Percentual_elegibilidade': '54.88'}, {'codigo_municipio': '1100106', 'nome_municipio': 'Guajar�-Mirim

# Aplicando Fuzzy em "num_idhm"

In [253]:
def aplica_fuzzy_idhm(df):
    # Variável de entrada
    IDH = ctrl.Antecedent(np.arange(0, 1.01, 0.01), 'IDH')
    # Variável de saída
    Classificacao = ctrl.Consequent(np.arange(0, 4, 1), 'Classificacao')

    # Conjuntos fuzzy para a variável de entrada
    IDH['Baixo'] = fuzz.trimf(IDH.universe, [0, 0, 0.5])
    IDH['Medio'] = fuzz.trimf(IDH.universe, [0.3, 0.5, 0.7])
    IDH['Alto'] = fuzz.trimf(IDH.universe, [0.5, 1, 1])

    # Conjuntos fuzzy para a variável de saída
    Classificacao['Baixo'] = fuzz.trimf(Classificacao.universe, [0, 0, 2])
    Classificacao['Medio'] = fuzz.trimf(Classificacao.universe, [1, 2, 3])
    Classificacao['Alto'] = fuzz.trimf(Classificacao.universe, [2, 3, 3])

    rule1 = ctrl.Rule(IDH['Baixo'], Classificacao['Baixo'])
    rule2 = ctrl.Rule(IDH['Medio'], Classificacao['Medio'])
    rule3 = ctrl.Rule(IDH['Alto'], Classificacao['Alto'])

    ctrl_sys = ctrl.ControlSystem([rule1, rule2, rule3])
    sim = ctrl.ControlSystemSimulation(ctrl_sys)

    list_ = []

    for index, row in df.iterrows():
        sim.input['IDH'] = row['num_idhm']
        sim.compute()
        list_.append({'codigo_municipio': f"{row['cod_mun']:.0f}", 'nome_municipio': f"{row['nom_mun']}", 'Percentual_elegibilidade': f"{sim.output['Classificacao']:.2f}"})
    return list_

In [254]:
num_idhm = cria_df_programa('num_idhm')

In [255]:
list_mun_elegibilidade_idhm = aplica_fuzzy_idhm(num_idhm)
print(list_mun_elegibilidade_idhm)

[{'codigo_municipio': '1100015', 'nome_municipio': "Alta Floresta D'Oeste", 'Percentual_elegibilidade': '2.07'}, {'codigo_municipio': '1100023', 'nome_municipio': 'Ariquemes', 'Percentual_elegibilidade': '2.59'}, {'codigo_municipio': '1100031', 'nome_municipio': 'Cabixi', 'Percentual_elegibilidade': '2.10'}, {'codigo_municipio': '1100049', 'nome_municipio': 'Cacoal', 'Percentual_elegibilidade': '2.60'}, {'codigo_municipio': '1100056', 'nome_municipio': 'Cerejeiras', 'Percentual_elegibilidade': '2.46'}, {'codigo_municipio': '1100064', 'nome_municipio': 'Colorado do Oeste', 'Percentual_elegibilidade': '2.37'}, {'codigo_municipio': '1100072', 'nome_municipio': 'Corumbiara', 'Percentual_elegibilidade': '2.06'}, {'codigo_municipio': '1100080', 'nome_municipio': 'Costa Marques', 'Percentual_elegibilidade': '2.06'}, {'codigo_municipio': '1100098', 'nome_municipio': "Espig�o D'Oeste", 'Percentual_elegibilidade': '2.24'}, {'codigo_municipio': '1100106', 'nome_municipio': 'Guajar�-Mirim', 'Perce

# Aplicando Fuzzy em "per_dom_vuln_var3"

In [256]:
def aplica_fuzzy_per_dom_vuln_var3(df):
    # Variável de entrada
    perc_dom_pobres = ctrl.Antecedent(np.arange(0, 101, 1), 'per_vuln_var3')
    # Variável de saída
    eligibilidade = ctrl.Consequent(np.arange(0, 101, 1), 'eligibilidade')

    # Definindo os conjuntos fuzzy para a variável de entrada
    perc_dom_pobres['baixissima'] = fuzz.trimf(perc_dom_pobres.universe, [0, 0, 5])
    perc_dom_pobres['baixa'] = fuzz.trimf(perc_dom_pobres.universe, [0, 6, 10])
    perc_dom_pobres['Mediana'] = fuzz.trimf(perc_dom_pobres.universe, [0, 11, 30])
    perc_dom_pobres['Alta'] = fuzz.trimf(perc_dom_pobres.universe, [0, 31, 70])
    perc_dom_pobres['Altíssima'] = fuzz.trimf(perc_dom_pobres.universe, [0, 71, 100])

    # Definindo os conjuntos fuzzy para a variável de saída
    eligibilidade['baixissima'] = fuzz.trimf(eligibilidade.universe, [0, 0, 5])
    eligibilidade['baixa'] = fuzz.trimf(eligibilidade.universe, [0, 6, 10])
    eligibilidade['Mediana'] = fuzz.trimf(eligibilidade.universe, [0, 11, 30])
    eligibilidade['Alta'] = fuzz.trimf(eligibilidade.universe, [0, 31, 70])
    eligibilidade['Altíssima'] = fuzz.trimf(eligibilidade.universe, [0, 71, 100])

    rule1 = ctrl.Rule(perc_dom_pobres['baixissima'], eligibilidade['baixissima'])
    rule2 = ctrl.Rule(perc_dom_pobres['baixa'], eligibilidade['baixa'])
    rule3 = ctrl.Rule(perc_dom_pobres['Mediana'], eligibilidade['Mediana'])
    rule4 = ctrl.Rule(perc_dom_pobres['Alta'], eligibilidade['Alta'])
    rule5 = ctrl.Rule(perc_dom_pobres['Altíssima'], eligibilidade['Altíssima'])

    ctrl_sys = ctrl.ControlSystem([rule1, rule2, rule3, rule4, rule5])
    sim = ctrl.ControlSystemSimulation(ctrl_sys)

    list_ = []
    for index, row in df.iterrows():
        sim.input['per_vuln_var3'] = row['per_dom_vuln_var3']
        sim.compute()
        list_.append({'codigo_municipio': f"{row['cod_mun']:.0f}", 'nome_municipio': f"{row['nom_mun']}", 'Percentual_elegibilidade': f"{sim.output['eligibilidade']:.2f}"})
    return list_

In [257]:
per_dom_vuln_var3 = cria_df_programa('per_dom_vuln_var3')

In [258]:
list_mun_per_dom_vuln_var3 = aplica_fuzzy_per_dom_vuln_var3(per_dom_vuln_var3)
print(list_mun_per_dom_vuln_var3)

[{'codigo_municipio': '1100015', 'nome_municipio': "Alta Floresta D'Oeste", 'Percentual_elegibilidade': '40.94'}, {'codigo_municipio': '1100023', 'nome_municipio': 'Ariquemes', 'Percentual_elegibilidade': '43.42'}, {'codigo_municipio': '1100031', 'nome_municipio': 'Cabixi', 'Percentual_elegibilidade': '43.79'}, {'codigo_municipio': '1100049', 'nome_municipio': 'Cacoal', 'Percentual_elegibilidade': '39.41'}, {'codigo_municipio': '1100056', 'nome_municipio': 'Cerejeiras', 'Percentual_elegibilidade': '41.59'}, {'codigo_municipio': '1100064', 'nome_municipio': 'Colorado do Oeste', 'Percentual_elegibilidade': '43.79'}, {'codigo_municipio': '1100072', 'nome_municipio': 'Corumbiara', 'Percentual_elegibilidade': '38.86'}, {'codigo_municipio': '1100080', 'nome_municipio': 'Costa Marques', 'Percentual_elegibilidade': '38.28'}, {'codigo_municipio': '1100098', 'nome_municipio': "Espig�o D'Oeste", 'Percentual_elegibilidade': '39.51'}, {'codigo_municipio': '1100106', 'nome_municipio': 'Guajar�-Mirim

# Aplicando Fuzzy em "perc_pobres_0a11_var2"

In [259]:
def aplica_fuzzy_perc_pobres_0a11_var2(df):
    # Variável de entrada
    perc_dom_pobres = ctrl.Antecedent(np.arange(0, 101, 1), 'pobres_0a11_var2')
    # Variável de saída
    eligibilidade = ctrl.Consequent(np.arange(0, 101, 1), 'eligibilidade')

    # Definindo os conjuntos fuzzy para a variável de entrada
    perc_dom_pobres['baixissima'] = fuzz.trimf(perc_dom_pobres.universe, [0, 0, 2])
    perc_dom_pobres['baixa'] = fuzz.trimf(perc_dom_pobres.universe, [0, 3, 10])
    perc_dom_pobres['Mediana'] = fuzz.trimf(perc_dom_pobres.universe, [0, 11, 20])
    perc_dom_pobres['Alta'] = fuzz.trimf(perc_dom_pobres.universe, [0, 21, 35])
    perc_dom_pobres['Altíssima'] = fuzz.trimf(perc_dom_pobres.universe, [0, 36, 100])

    # Definindo os conjuntos fuzzy para a variável de saída
    eligibilidade['baixissima'] = fuzz.trimf(eligibilidade.universe, [0, 0, 2])
    eligibilidade['baixa'] = fuzz.trimf(eligibilidade.universe, [0, 3, 10])
    eligibilidade['Mediana'] = fuzz.trimf(eligibilidade.universe, [0, 11, 20])
    eligibilidade['Alta'] = fuzz.trimf(eligibilidade.universe, [0, 21, 35])
    eligibilidade['Altíssima'] = fuzz.trimf(eligibilidade.universe, [0, 36, 100])

    rule1 = ctrl.Rule(perc_dom_pobres['baixissima'], eligibilidade['baixissima'])
    rule2 = ctrl.Rule(perc_dom_pobres['baixa'], eligibilidade['baixa'])
    rule3 = ctrl.Rule(perc_dom_pobres['Mediana'], eligibilidade['Mediana'])
    rule4 = ctrl.Rule(perc_dom_pobres['Alta'], eligibilidade['Alta'])
    rule5 = ctrl.Rule(perc_dom_pobres['Altíssima'], eligibilidade['Altíssima'])

    ctrl_sys = ctrl.ControlSystem([rule1, rule2, rule3, rule4, rule5])
    sim = ctrl.ControlSystemSimulation(ctrl_sys)

    list_ = []
    for index, row in df.iterrows():
        sim.input['pobres_0a11_var2'] = row['perc_pobres_0a11_var2']
        sim.compute()
        list_.append({'codigo_municipio': f"{row['cod_mun']:.0f}", 'nome_municipio': f"{row['nom_mun']}", 'Percentual_elegibilidade': f"{sim.output['eligibilidade']:.2f}"})
    return list_

In [260]:
perc_pobres_0a11_var2 = cria_df_programa('perc_pobres_0a11_var2')

In [261]:
list_mun_perc_pobres_0a11_var2 = aplica_fuzzy_perc_pobres_0a11_var2(perc_pobres_0a11_var2)
print(list_mun_perc_pobres_0a11_var2)

[{'codigo_municipio': '1100015', 'nome_municipio': "Alta Floresta D'Oeste", 'Percentual_elegibilidade': '47.48'}, {'codigo_municipio': '1100023', 'nome_municipio': 'Ariquemes', 'Percentual_elegibilidade': '45.93'}, {'codigo_municipio': '1100031', 'nome_municipio': 'Cabixi', 'Percentual_elegibilidade': '47.21'}, {'codigo_municipio': '1100049', 'nome_municipio': 'Cacoal', 'Percentual_elegibilidade': '45.35'}, {'codigo_municipio': '1100056', 'nome_municipio': 'Cerejeiras', 'Percentual_elegibilidade': '46.50'}, {'codigo_municipio': '1100064', 'nome_municipio': 'Colorado do Oeste', 'Percentual_elegibilidade': '45.79'}, {'codigo_municipio': '1100072', 'nome_municipio': 'Corumbiara', 'Percentual_elegibilidade': '47.74'}, {'codigo_municipio': '1100080', 'nome_municipio': 'Costa Marques', 'Percentual_elegibilidade': '47.89'}, {'codigo_municipio': '1100098', 'nome_municipio': "Espig�o D'Oeste", 'Percentual_elegibilidade': '46.87'}, {'codigo_municipio': '1100106', 'nome_municipio': 'Guajar�-Mirim